In [1]:
import csv
import numpy as np
import pandas as pd

In [2]:
#https://www.census.gov/data/datasets/time-series/demo/popest/2010s-state-total.html

In [2]:
# EDIT THIS TO CONTAIN ALL YEARS YOU WANT TO ANALYZE
years = ['2000', '2001', '2002', '2003', '2004',
         '2005', '2006', '2007', '2008', '2009',
         '2010', '2011', '2012',
         '2013', '2013', '2014', '2015', '2016', 
         '2017', '2018', '2019']

In [3]:
def drop_cols(col_names, df):
    for col_name in col_names:
        if col_name in df.columns:
            df = df.drop(columns=col_name)
    return df

In [8]:
def combine(year):
    unemployment_csv = f'./data/{year}_unemployment.csv'
    education_csv = f'./data/education_{year}.csv'
    income_csv = f'./data/yearly-income-{year}.csv'
    jobgrowth_csv = f'./data/{year}-jobgrowth.csv'
    
    if int(year) < 2011:
        population_csv = f'./data/nst-est2000-alldata.csv'
    else:
        population_csv = f'./data/nst-est2019-alldata.csv'
    snaps_csv = f'./data/snaps_data_{year}.csv'
    unwanted_cols = ['rank', 'Rank', 'labor force', 'Labor Force', 'Rank ']
    
    unemployment_df = pd.read_csv(unemployment_csv)
    education_df = pd.read_csv(education_csv)
    pop_df = pd.read_csv(population_csv)
    
    income_df = pd.read_csv(income_csv)
    income_df['State'] = income_df['State'].str.title()
    
    jobgr_df = pd.read_csv(jobgrowth_csv)
    
    #print(pop_df.head())
    #print(population_csv)

    pop_df = pop_df[["NAME", f"POPESTIMATE{year}"]].copy()
    snaps_df = pd.read_csv(snaps_csv)
    education_df = education_df.rename(columns={"State or Other Area": "State", f"{year} Actual": "Education Budget"})
    unemployment_df = unemployment_df.rename(columns={"state": "State", f"{year} rate": "Unemployment Rate", f"{year} Rate": "Unemployment Rate"})
    #print(unemployment_df)
    pop_df = pop_df.rename(columns={"NAME": "State", f"POPESTIMATE{year}": "Population Estimate"})
    snaps_df = snaps_df.rename(columns = {"state": "State", "household_participants": "SNAPS household_participants",
                                   "individual_participants": "SNAPS individual_participants", "cost": "SNAPS cost",
                                   "cost_per_person": "SNAPS cost_per_person", "cost_per_household": "SNAPS cost_per_household"})
    
    df = pd.merge(education_df, unemployment_df, on='State')
    df = pd.merge(df, pop_df, how='left', on='State')
    df = pd.merge(df, snaps_df, how='left', on='State')
    df = pd.merge(df, income_df, how='left', on='State')
    df = pd.merge(df, jobgr_df, how='left', on='State')
    if year == '2019':
        print(df)
    df = drop_cols(unwanted_cols, df)
    
    print(year)
    print(df.columns)
    
    return df

In [9]:
for year in years:
    df = combine(year)
#     print(df.head())
    df.to_csv(f'./data/{year}-combined.csv', index=False)

#df.head()

2000
Index(['State', 'Education Budget', 'Unemployment Rate', 'Population Estimate',
       'SNAPS household_participants', 'SNAPS individual_participants',
       'SNAPS cost', 'SNAPS cost_per_person', 'SNAPS cost_per_household',
       'DATE', 'Median Income', 'Year', 'Unnamed: 0', 'U.S. Rank', '% Change',
       'Job Growth (In Thousands)', '# of Jobs (In Thousands)'],
      dtype='object')
2001
Index(['State', 'Education Budget', 'Unemployment Rate', 'Population Estimate',
       'SNAPS household_participants', 'SNAPS individual_participants',
       'SNAPS cost', 'SNAPS cost_per_person', 'SNAPS cost_per_household',
       'DATE', 'Median Income', 'Year', 'Unnamed: 0', 'U.S. Rank', '% Change',
       'Job Growth (In Thousands)', '# of Jobs (In Thousands)'],
      dtype='object')
2002
Index(['State', 'Education Budget', 'Unemployment Rate', 'Population Estimate',
       'SNAPS household_participants', 'SNAPS individual_participants',
       'SNAPS cost', 'SNAPS cost_per_person', 'SN

In [21]:
df = ''

In [10]:
for year in years:
    source = f'./data/{year}-combined.csv'
    ydf = pd.read_csv(source)
    ydf['Year'] = year
    if year != '2000':
        df = pd.concat([df, ydf], ignore_index=True)
    else: 
        df = ydf.copy()
    



In [13]:
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

df.to_csv('./data/data.csv', index=False)
print(len(df))
df.head(-45)

1051


,State,Education Budget,Unemployment Rate,Population Estimate,SNAPS household_participants,SNAPS individual_participants,SNAPS cost,SNAPS cost_per_person,SNAPS cost_per_household,DATE,Median Income,Year,U.S. Rank,% Change,Job Growth (In Thousands),# of Jobs (In Thousands)
0,Alabama,5.577678e+08,4.6,4447207,1.561046e+05,3.960572e+05,3.437353e+08,72.3244,183.4962,2000-01-01,35424.0,2000,49.0,1.03,19.8,1945.1
1,Alaska,1.778816e+08,6.6,626933,1.320833e+04,3.752433e+04,4.583456e+07,101.7885,289.1770,2000-01-01,83377.0,2000,40.0,1.75,4.7,273.0
2,Arizona,6.679765e+08,3.9,5130247,9.556750e+04,2.590026e+05,2.402456e+08,77.2983,209.4903,2000-01-01,62766.0,2000,1.0,4.48,96.1,2239.8
3,Arkansas,3.284035e+08,4.4,2673293,9.876433e+04,2.465715e+05,2.062364e+08,69.7013,174.0139,2000-01-01,46853.0,2000,32.0,2.36,26.7,1157.6
4,California,3.792097e+09,4.9,33871653,6.720070e+05,1.830677e+06,1.639333e+09,74.6233,203.2882,2000-01-01,73862.0,2000,7.0,3.58,498.5,14421.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1001,Alaska,4.005232e+08,5.6,731545,3.549300e+04,7.893200e+04,1.710917e+08,401.7000,180.6300,2019-01-01,83079.0,2019,41.0,0.54,1.7,316.4
1002,Arizona,5.143402e+09,4.8,7278717,3.451060e+05,7.368830e+05,1.146712e+09,276.9000,129.6800,2019-01-01,74897.0,2019,3.0,2.93,83.2,2922.7
1003,Arkansas,1.350409e+09,3.5,3017804,1.592310e+05,3.549170e+05,4.603336e+08,240.9200,108.0800,2019-01-01,57799.0,2019,14.0,1.63,20.5,1275.0
1004,California,1.721222e+10,4.1,39512223,1.803402e+06,3.529470e+06,5.977281e+09,276.2000,141.1300,2019-01-01,82772.0,2019,15.0,1.50,255.1,17250.8
